# House Sales Price

<br>
<img src="img/house_sale.png">
<br>

Os preços de venda de casas de conjunto de dados para King County, Seattle. Inclui casas vendidas entre maio de 2014 e maio de 2015. O conjunto de dados fornece recursos que as casas têm e o preço pelo qual elas foram vendidas. Pode ser usado para modelar a previsão de preços de casas.

O conjunto de dados está disponível em kaggle [House Sales in King County, USA](https://www.kaggle.com/harlfoxem/housesalesprediction)

Alguns dos atributos capturados no conjunto de dados são

- Número de quartos e banheiros
- Total de pés quadrados 
- Quantos andares
- Se tem um porão e tamanho do porão
- Classificação da casa
- Clima tem beira-mar e a qualidade da vista
- Quando a casa foi construída e se a casa é renovada, se for renovado
- Latitude e longitude
- preço da casa
- Quando a casa foi vendida

Perguntas:


- Quais as 10 casas com a melhor relação custo/benefício do preço por metro quadrado?
- Qual a correlação do nível de classificação as casas em relação ao seu preço?
- Podemos afirmar que estar de frente para o lago representa um fator que eleva o preço médio das casas, com uma confiança superior a 95%?

Levante algumas hipóteses a sua escolha e comprove se é possível não-rejeitar ou rejeitar, respondendo se as médias são estatisticamente iguais/equivalentes ou estatisticamente diferentes/significativas.


## Lendo e conhecendo o dataset

In [ ]:
# improtando as bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# lendo e armazenando os dados
#kc = pd.read_csv(r'../../../data/kc_house_data.csv')

In [ ]:
# lendo e verificando os dados
%time kc = pd.read_csv("http://www.dropbox.com/s/17lhj34voz3bbz7/kc_house_data.csv?dl=1") 

In [ ]:
kc.head()   

In [ ]:
# informações básicas do DataSet
kc.info()

In [ ]:
# verificando a distribuição dos dados de preço das casas
kc.price.hist(bins=100)

In [ ]:
# plotando uma dispersão dos dados de preços das casas
plt.scatter(x=kc.id, y=kc.price, alpha=0.5)
plt.show()

## Quais as 10 casas com a melhor relação custo/benefício do preço por metro quadrado?

In [ ]:
# plotando uma dispersão dos preços das casas pea área
import seaborn as sns

sns.jointplot(x="sqft_living", y="price", data=kc, kind = 'reg',fit_reg= True, size = 7)
plt.show()

In [ ]:
# criando cuma coluna de preços por pe's quadrados
kc['price_per_sqft'] = kc.price / kc.sqft_living
kc['price_per_sqft'].hist(bins=100)

In [ ]:
# selecioando os dez valores com menor relação de preço por pé quadrado
kc['price_per_sqft'].sort_values()[:10]

A resposta é o índice das casas que estão mostradas no filtro acima: estas são as casas com melhor custo/benefício em relação à sua área.

In [ ]:
# selecioando as dez casas com menor relação de preço por pé quadrado
kc['price_per_sqft'].sort_values()[:10].index.values

## Qual a correlação do nível de classificação as casas em relação ao seu preço?

In [ ]:
# contagem dos níveis de classificação das casas
kc.grade.value_counts()

In [ ]:
# dispersão dos preços das casas pela classificação
sns.jointplot(x="grade", y="price", data=kc, kind = 'reg',fit_reg= True, size = 7)
plt.show()

In [ ]:
# cálculo da correlação entre os preços das casas e a classificação
np.corrcoef(x=kc.grade, y=kc.price)

## Podemos afirmar que estar de frente para o lago representa um fator que eleva o preço médio das casas, com uma confiança superior a 95%?

In [ ]:
# informações básicas do DataFrame
kc.info()

In [ ]:
# contagem do número de casas de frente pra água
kc.waterfront.value_counts()

In [ ]:
# cálculo da média e desvio-padrão das casas de frente e não de frente pra água
import scipy.stats as stats

nwf_mu = kc[kc['waterfront'] == 0]['price'].mean()
nwf_sigma = kc[kc['waterfront'] == 0]['price'].std()

ywf_mu = kc[kc['waterfront'] == 1]['price'].mean()
ywf_sigma = kc[kc['waterfront'] == 1]['price'].std()

In [ ]:
# plotagem das duas curvas normais de distribuição do preço das casas de frente e não de frente pra água
x = np.linspace(nwf_mu - 3*ywf_sigma, ywf_mu + 3*ywf_sigma, 100)

fig, ax = plt.subplots(1, 1)
ax.plot(x, stats.norm.pdf(x, nwf_mu, nwf_sigma),label='waterfront')
ax.plot(x, stats.norm.pdf(x, ywf_mu, ywf_sigma),label='no-waterfront')
ax.legend(loc='best', frameon=False)


In [ ]:
# intervalo de confiança do preço das casas não de frente pra água
stats.norm.interval(0.95, loc=nwf_mu, scale=nwf_sigma)

In [ ]:
# média do preço das casas de frente pra água
ywf_mu

In [ ]:
# inferência estatístiva da média do preço das casas de frente pra água  
# em comparação com a média do preço das casas não de frente pra água
stats.norm.interval(0.95, loc=nwf_mu, scale=nwf_sigma)[0] < ywf_mu < stats.norm.interval(0.95, loc=nwf_mu, scale=nwf_sigma)[1]

Como a média das casas de frente para a água não está no intervalo de confiança da distribuição normal das casas que não estão de frente para a água, podemos afirmar que estar de frente para a água eleva o preço das casas, com 95% de confiança.

In [ ]:
# teste t de inferência para saber de a média do preço das casas não de frente pra água 
# é equivalente à média do preço das casas de frente pra água

a = kc[kc['waterfront'] == 0]['price']
popmean = ywf_mu
stats.ttest_1samp(a=a, popmean=popmean)

Como o p_value é menos que 0.05, isso significa da a média do preço das casas não de frente pra água é diferente da média do preço das casas de frente pra água. 


Como o o valor do Statistic é a diferença entre a média do preço das casas da série de dados"a" (casas não de frente pra água) e o valor do "popmean" (média do preço das casas de frente pra água).

### t-test para as médias de duas séries 

In [ ]:
# teste de inferência para a média das duas séries 
# Statistic = media_a - media_b
stats.ttest_ind(a=kc[kc['waterfront'] == 0]['price'],
                b=kc[kc['waterfront'] == 1]['price'], 
                equal_var = False)

Para um p-value menor que 0.05, as médias das duas séries são estatísticamente diferentes! Isto significa que o preço médio das casas de frente para a água é estatísticamente superior ao preço médio das casas que não estão de frente para as casas, com 95% de confiança. 


Como o valor do teste Statistic é negativo, isso significa que a média das casas da série a (preço das casas não de frente pra água) é menor que a média de b (preço das casas de frente pra água).